In [13]:
import pandas as pd
import numpy as np
import csv
import os
import torch
import pickle
from torch.utils.data import Dataset,DataLoader,TensorDataset
from tqdm import tqdm
import models.Multi_view_multi_task_ablition_study
from torch import nn, optim
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn as nn
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, average_precision_score,precision_score,f1_score,recall_score
from scipy.stats import rankdata

In [14]:
torch.cuda.device_count()
m_gpu=1
os.environ['CUDA_VISIBLE_DEVICES'] = '%d' % m_gpu
torch.cuda.set_device(m_gpu)
torch.cuda.is_available()
torch.cuda.current_device()

1

In [15]:
seed=3447
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) 

In [16]:
namedataset="Helpdesk"
# namedataset="BPI_Challenge_2012_W_Complete"
# namedataset="BPI_Challenge_2012_W"
# namedataset="BPI_Challenge_2012_A"
# namedataset="BPI_Challenge_2012_O"

In [17]:
results=[]
for step in range(5):
    fold="fold"+str(step)
    train_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/train_"+namedataset+".csv"
    val_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/val_"+namedataset+".csv"
    test_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/test_"+namedataset+".csv"
    df_test=pd.read_csv(test_file)
    alpha=0.3
    fold="fold"+str(step)
    pattern_act_dict=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/pattern_act_dict_"+str(alpha)+".pkl","rb"))
    pattern_act_time_dict=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/pattern_act_time_dict_"+str(alpha)+".pkl","rb"))
    pattern_act_res_dict=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/pattern_act_res_dict_"+str(alpha)+".pkl","rb"))
    res_cluster_label=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/res_cluster_label_"+str(alpha)+".pkl","rb"))
    res_cluster_num=max(res_cluster_label)+1

    pattern_act_sup_list=[]
    for key in pattern_act_dict:
        # print(len(pattern_act_dict[key]))
        pattern_act_sup_list.append(len(pattern_act_dict[key]))

    pattern_act_res_sup_list=[]
    for key in pattern_act_res_dict:
        # print(len(pattern_act_dict[key]))
        pattern_act_res_sup_list.append(len(pattern_act_res_dict[key]))

    pattern_act_time_sup_list=[]
    for key in pattern_act_time_dict:
        # print(len(pattern_act_dict[key]))
        pattern_act_time_sup_list.append(len(pattern_act_time_dict[key]))

    try:
        df_test.columns = ["CaseID", "Activity", "Resource", "Timestamp","amount"]
    except:
        df_test.columns = ["CaseID", "Activity", "Resource", "Timestamp"]

    df_train=pd.read_csv(train_file)
    try:
        df_train.columns = ["CaseID", "Activity", "Resource", "Timestamp","amount"]
    except:
        df_train.columns = ["CaseID", "Activity", "Resource", "Timestamp"]

    df_valid=pd.read_csv(val_file)
    try:
        df_valid.columns = ["CaseID", "Activity", "Resource", "Timestamp","amount"]
    except:
        df_valid.columns = ["CaseID", "Activity", "Resource", "Timestamp"]

    df_train.fillna(1, inplace=True)
    df_test.fillna(1,inplace=True)
    df_valid.fillna(1,inplace=True)

    cont_trace = df_train['CaseID'].value_counts(dropna=False)
    cont_test_trace = df_test['CaseID'].value_counts(dropna=False)
    cont_val_trace = df_valid['CaseID'].value_counts(dropna=False)

    max_trace = max(cont_trace)
    max_test_trace = max(cont_test_trace)
    max_val_trace = max(cont_val_trace)
    real_max=max(max_trace,max_test_trace,max_val_trace)
    df_full=df_train.append(df_test).append(df_valid)
    mean_trace=int(round(np.mean(df_full['CaseID'].value_counts(dropna=False))))
    
    test_listOfResource=df_test["Resource"].unique().tolist()
    train_listOfResource=df_train["Resource"].unique().tolist()
    valid_listOfResource=df_valid["Resource"].unique().tolist()

    test_resourceset=set(test_listOfResource)
    train_resourceset=set(train_listOfResource)
    valid_resourceset=set(valid_listOfResource)
    all_resource=list(valid_resourceset.union(test_resourceset.union(train_resourceset)))
    all_resource.sort()
    listOfresourcesInt = list(range(0, len(all_resource)))
    resourceMapping=dict(zip(all_resource,listOfresourcesInt))

    test_listOfEvents=df_test["Activity"].unique().tolist()
    train_listOfEvents=df_train["Activity"].unique().tolist()
    valid_listOfEvents=df_valid["Activity"].unique().tolist()
    test_eventset=set(test_listOfEvents)
    train_eventset=set(train_listOfEvents)
    valid_eventset=set(valid_listOfEvents)
    all_events=list(valid_eventset.union(test_eventset.union(train_eventset)))
    all_events.sort()
    listOfeventsInt = list(range(0, len(all_events)))
    mapping = dict(zip(all_events, listOfeventsInt))

    res_cluster_label=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/res_cluster_label_"+str(alpha)+".pkl","rb"))
    res_cluster_num=max(res_cluster_label)+1
    remain_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/time_normlize.pkl","rb"))
    real_remain=max(remain_list)
        
    train_data=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_data_"+str(alpha)+".pkl","rb"))
    train_label=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_label_"+str(alpha)+".pkl","rb"))
    train_act_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_act_pattern_list_"+str(alpha)+".pkl","rb"))
    train_act_res_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_act_res_pattern_list_"+str(alpha)+".pkl","rb"))
    train_act_time_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_act_time_pattern_list_"+str(alpha)+".pkl","rb"))

    test_data=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_data_"+str(alpha)+".pkl","rb"))
    test_label=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_label_"+str(alpha)+".pkl","rb"))
    test_act_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_act_pattern_list_"+str(alpha)+".pkl","rb"))
    test_act_res_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_act_res_pattern_list_"+str(alpha)+".pkl","rb"))
    test_act_time_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_act_time_pattern_list_"+str(alpha)+".pkl","rb"))

    valid_data=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_data_"+str(alpha)+".pkl","rb"))
    valid_label=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_label_"+str(alpha)+".pkl","rb"))
    valid_act_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_act_pattern_list_"+str(alpha)+".pkl","rb"))
    valid_act_res_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_act_res_pattern_list_"+str(alpha)+".pkl","rb"))
    valid_act_time_pattern_list=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_act_time_pattern_list_"+str(alpha)+".pkl","rb"))

    num_resource=len(all_resource)+1
    num_class=len(all_events)+1

    res_embsize=(num_resource+1)//2
    act_embsize=(num_class+1)//2

    train_data=torch.tensor(np.array(train_data),dtype=torch.float32).cuda()
    train_label=torch.tensor(np.array(train_label),dtype=torch.float32).cuda()
    test_data=torch.tensor(np.array(test_data),dtype=torch.float32).cuda()
    test_label=torch.tensor(np.array(test_label),dtype=torch.float32).cuda()
    valid_data=torch.tensor(np.array(valid_data),dtype=torch.float32).cuda()
    valid_label=torch.tensor(np.array(valid_label),dtype=torch.float32).cuda()

    
    train_act_pattern=torch.tensor(np.array(train_act_pattern_list),dtype=torch.long).cuda()
    train_act_res_pattern=torch.tensor(np.array(train_act_res_pattern_list),dtype=torch.long).cuda()
    train_act_time_pattern=torch.tensor(np.array(train_act_time_pattern_list),dtype=torch.long).cuda()
    valid_act_pattern=torch.tensor(np.array(valid_act_pattern_list),dtype=torch.long).cuda()
    valid_act_res_pattern=torch.tensor(np.array(valid_act_res_pattern_list),dtype=torch.long).cuda()
    valid_act_time_pattern=torch.tensor(np.array(valid_act_time_pattern_list),dtype=torch.long).cuda()
    test_act_pattern=torch.tensor(np.array(test_act_pattern_list),dtype=torch.long).cuda()
    test_act_res_pattern=torch.tensor(np.array(test_act_res_pattern_list),dtype=torch.long).cuda()
    test_act_time_pattern=torch.tensor(np.array(test_act_time_pattern_list),dtype=torch.long).cuda()

    batch_size=128
    train_dataset=TensorDataset(train_data,train_label[:,0],train_label[:,1],train_label[:,5],train_act_pattern,train_act_res_pattern,train_act_time_pattern)
    train_dataloader=DataLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                shuffle=True
                                )
    valid_dataset=TensorDataset(valid_data,valid_label[:,0],valid_label[:,1],valid_label[:,5],valid_act_pattern,valid_act_res_pattern,valid_act_time_pattern)
    valid_dataloader=DataLoader(dataset=valid_dataset,
                                batch_size=batch_size,
                                shuffle=True
                                )
    
    test_dataset=TensorDataset(test_data,test_label[:,0],test_label[:,1],test_label[:,5],test_act_pattern,test_act_res_pattern,test_act_time_pattern)
    test_dataloader=DataLoader(dataset=test_dataset,
                            batch_size=batch_size,
                            shuffle=True
                            )

    hidden_dim=20
    embsize_list=[act_embsize,res_embsize]
    catnum_list=[num_class,num_resource]
    num_size=2+7+24
    num_timestep=train_data.shape[1]
    embsize_list=[act_embsize,res_embsize]
    num_list=[num_class,num_resource]
    num_layers=3
    kernel_size=2
    dropout=0.3
    pattern_embsize_list=[act_embsize,32,32]
    pattern_num_list=[num_class,res_cluster_num,10]
                        
    model=models.Multi_view_multi_task_ablition_study.Multi_view_multi_task_no_task_relevance(embsize_list,num_list,hidden_dim,kernel_size,dropout,num_layers,pattern_embsize_list,pattern_num_list).cuda()
    
    class MTLLoss(nn.Module):
        def __init__(self, task_nums):
            super(MTLLoss, self).__init__()
            x = torch.zeros([task_nums], dtype=torch.float32)
            self.log_var2s = nn.Parameter(x)
        def forward(self,loss_list):
            loss = 0
            for i in range(len(self.log_var2s)):
                # mse = (logit_list[i] - label_list[i]) ** 2
                pre = torch.exp(-self.log_var2s[i])
                loss += torch.sum(pre * loss_list[i] + self.log_var2s[i], dim=-1)
            return torch.mean(loss)
    lr=1e-3
    my_optim="Adam"
    criterion = nn.CrossEntropyLoss()
    mae=nn.L1Loss()
    mtlloss=MTLLoss(3).cuda()
    # 定义GradNorm参数
    task_numbers = 3 # 任务数
    mu = 1e-2 # GradNorm参数
    optimizer = getattr(optim, my_optim)([{"params":model.parameters()},{"params":mtlloss.parameters()}], lr=lr)
    ExpLR = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    bestloss=1e6

    def test(ttest_dataLoader):
        model.eval()
        tloss=0
        
        act_pred=np.array([])
        act_truth=np.array([])

        res_pred=np.array([])
        res_truth=np.array([])
        
        time_pred=np.array([])
        time_truth=np.array([])

        test_data=np.array([])
        test_caseid=np.array([])

        test_length=np.array([])

        for j,(data,ylabels,res_labels,time_labels,act_pattern,act_res_pattern,act_time_pattern) in enumerate(ttest_dataLoader):
            optimizer.zero_grad()
            actoutput,resoutput,remain = model(data,act_pattern,act_res_pattern,act_time_pattern)
            # actoutput,resoutput,remain = model(data)

            # actoutput,timeoutput,resoutput,remain_output = model(data,trans_prob)

            # loss1 = criterion(actoutput.squeeze(dim=2), label.long())
            loss1 = criterion(actoutput, ylabels.long().reshape(data.size(0)))
            loss2 = criterion(resoutput,res_labels.long().reshape(data.size(0)))
            loss3 = mae(remain,time_labels.reshape(data.size(0),1))
            # ExpLR.step()
            tloss+=(loss3.item()*data.shape[0])
            # (0.5*loss1+0.5*loss2).backward()
            # optimizer.step()
            softmaxoutput=nn.functional.softmax(actoutput,dim=1)
            predicted=torch.argmax(softmaxoutput,dim=1)
            groundTruth=ylabels
            res_predicted=torch.argmax(resoutput,dim=1)
            # res_truth=label[1]

            act_pred=np.append(act_pred,predicted.cpu().numpy())
            act_truth=np.append(act_truth,groundTruth.cpu().numpy())

            res_pred=np.append(res_pred,res_predicted.cpu().numpy())
            res_truth=np.append(res_truth,res_labels.cpu().numpy())

            time_pred=np.append(time_pred,remain.detach().cpu().numpy())
            time_truth=np.append(time_truth,time_labels.cpu().numpy())
            
            test_data=np.append(test_data,data[:,:,0].cpu().numpy()) 

            # test_length=np.append(test_length,length_labels.cpu().numpy())

        return act_pred,act_truth,res_pred,res_truth,time_pred,time_truth,_,tloss

    def valid(ttest_dataLoader):
        model.eval()
        tloss=0
        
        act_pred=np.array([])
        act_truth=np.array([])

        res_pred=np.array([])
        res_truth=np.array([])
        
        time_pred=np.array([])
        time_truth=np.array([])

        test_data=np.array([])
        test_caseid=np.array([])

        test_length=np.array([])

        for j,(data,ylabels,res_labels,time_labels,act_pattern,act_res_pattern,act_time_pattern) in enumerate(ttest_dataLoader):
            optimizer.zero_grad()
            actoutput,resoutput,remain = model(data,act_pattern,act_res_pattern,act_time_pattern)
            # actoutput,resoutput,remain = model(data)

            # actoutput,timeoutput,resoutput,remain_output = model(data,trans_prob)

            # loss1 = criterion(actoutput.squeeze(dim=2), label.long())
            loss1 = criterion(actoutput, ylabels.long().reshape(data.size(0)))
            loss2 = criterion(resoutput,res_labels.long().reshape(data.size(0)))
            loss3 = mae(remain,time_labels.reshape(data.size(0),1))
            # ExpLR.step()
            tloss+=(loss3.item()*data.shape[0])
            # (0.5*loss1+0.5*loss2).backward()
            # optimizer.step()
            softmaxoutput=nn.functional.softmax(actoutput,dim=1)
            predicted=torch.argmax(softmaxoutput,dim=1)
            groundTruth=ylabels
            res_predicted=torch.argmax(resoutput,dim=1)
            # res_truth=label[1]

            act_pred=np.append(act_pred,predicted.cpu().numpy())
            act_truth=np.append(act_truth,groundTruth.cpu().numpy())

            res_pred=np.append(res_pred,res_predicted.cpu().numpy())
            res_truth=np.append(res_truth,res_labels.cpu().numpy())

            time_pred=np.append(time_pred,remain.detach().cpu().numpy())
            time_truth=np.append(time_truth,time_labels.cpu().numpy())
            
            test_data=np.append(test_data,data[:,:,0].cpu().numpy()) 

            # test_length=np.append(test_length,length_labels.cpu().numpy())

        return act_pred,act_truth,res_pred,res_truth,time_pred,time_truth,_,tloss

    def cal_accuracy_precision_f1score(y_true,y_pred):
        recall=recall_score(y_true,y_pred,average="weighted",zero_division=1)
        precision=precision_score(y_true,y_pred,average="weighted",zero_division=1)
        f1score=f1_score(y_true,y_pred,average="weighted",zero_division=1)
        return recall,precision,f1score
    
    def cal_mse(y_true,y_pred):
        y_pred=y_pred*(real_remain/(24*60*60))
        y_true=y_true*(real_remain/(24*60*60))
        return np.mean((y_true - y_pred) ** 2)
    def cal_mae(y_true,y_pred):
        y_pred=y_pred*(real_remain/(24*60*60))
        y_true=y_true*(real_remain/(24*60*60))
        return np.mean(np.abs(y_pred-y_true))

    def train(epoch):
        valid_act_acc_list=[]
        valid_act_pre_list=[]
        valid_act_fscore_list=[]
        valid_res_acc_list=[]
        valid_res_pre_list=[]
        valid_res_fscore_list=[]
        valid_time_mae_list=[]
        valid_time_mse_list=[]
        test_act_acc_list=[]
        test_act_pre_list=[]
        test_act_fscore_list=[]
        test_res_acc_list=[]
        test_res_pre_list=[]
        test_res_fscore_list=[]
        test_time_mae_list=[]
        test_time_mse_list=[]

        for m in tqdm(range(epoch)):
            print("epoch  ",m)
            model.train()
            t_loss=[]
            trainpred=np.array([])
            traintruth=np.array([])
            train_respred=np.array([])
            train_restruth=np.array([])
            # print(gradnormloss.w)
            for j,(data,label,reslabel,timelabel,act_pattern,act_res_pattern,act_time_pattern) in enumerate(train_dataloader):

                optimizer.zero_grad()
                actoutput,resoutput,timeoutput = model(data,act_pattern,act_res_pattern,act_time_pattern)
                # actoutput,resoutput,timeoutput = model(data)

                # loss1 = criterion(actoutput.squeeze(dim=2), label.long())
                loss1 = criterion(actoutput, label.long().reshape(data.size(0)))
                loss3 = mae(timeoutput,timelabel.reshape(data.size(0),1))
                loss2 = criterion(resoutput, reslabel.long().reshape(data.size(0)))
        
                # paraments=model.get_last_layer_shared()
                # gradnormloss(torch.cat((loss1,loss2),dim=1),paraments,optimizer,None)
                # (loss1*0.2+loss2*1.8).backward()
                # (loss1*0.5+loss3*0.5).backward()
                # (0.7*loss1+0.2*loss2+0.1*loss3).backward()
                # loss1.backward()
                loss=mtlloss([loss1,loss2,loss3])
                loss.backward()
                # loss3.backward()
                optimizer.step()
                t_loss.append([loss1.item(),loss2.item(),loss3.item()])
                softmaxoutput=nn.functional.softmax(actoutput,dim=1)
                predicted=torch.argmax(softmaxoutput,dim=1)
                groundTruth=label
                trainpred=np.append(trainpred,predicted.cpu().numpy())
                traintruth=np.append(traintruth,groundTruth.cpu().numpy())

                res_predicted=torch.argmax(resoutput,dim=1)
                res_groundTruth=reslabel
                train_respred=np.append(train_respred,res_predicted.cpu().numpy())
                train_restruth=np.append(train_restruth,res_groundTruth.cpu().numpy()) 
            scheduler.step()
            model.eval()

            act_pred,act_truth,res_pred,res_truth,time_pred,time_truth,test_length,tloss=test(test_dataloader)
            val_act_pred,val_act_truth,val_res_pred,val_res_truth,val_time_pred,val_time_truth,test_length,tloss=valid(valid_dataloader)

            test_act_acc,test_act_pre,test_act_fscore=cal_accuracy_precision_f1score(act_truth,act_pred)
            test_res_acc,test_res_pre,test_res_fscore=cal_accuracy_precision_f1score(res_truth,res_pred)
            time_mae=cal_mae(time_truth,time_pred)
            time_mse=cal_mse(time_truth,time_pred)

            valid_act_acc,valid_act_pre,valid_act_fscore=cal_accuracy_precision_f1score(val_act_truth,val_act_pred)
            valid_res_acc,valid_res_pre,valid_res_fscore=cal_accuracy_precision_f1score(val_res_truth,val_res_pred)
            valid_time_mae=cal_mae(val_time_truth,val_time_pred)
            valid_time_mse=cal_mse(val_time_truth,val_time_pred)

            valid_act_acc_list.append(valid_act_acc)
            valid_act_pre_list.append(valid_act_pre)
            valid_act_fscore_list.append(valid_act_fscore)
            valid_res_acc_list.append(valid_res_acc)
            valid_res_pre_list.append(valid_res_pre)
            valid_res_fscore_list.append(valid_res_fscore)
            valid_time_mae_list.append(valid_time_mae)
            valid_time_mse_list.append(valid_time_mse)


            test_act_acc_list.append(test_act_acc)
            test_act_pre_list.append(test_act_pre)
            test_act_fscore_list.append(test_act_fscore)
            test_res_acc_list.append(test_res_acc)
            test_res_pre_list.append(test_res_pre)
            test_res_fscore_list.append(test_res_fscore)
            test_time_mae_list.append(time_mae)
            test_time_mse_list.append(time_mse)
        
        def rank(score,mode=1):
            score=np.array(score)
            ranks=rankdata(score)
            if mode==1:
                return ranks
            else:
                ranks=len(score)+1-ranks
                return ranks

        act_acc_rank=rank(valid_act_acc_list)
        act_pre_rank=rank(valid_act_pre_list)
        act_fscore_rank=rank(valid_act_fscore_list)
        res_acc_rank=rank(valid_res_acc_list)
        res_pre_rank=rank(valid_res_pre_list)
        res_fscore_rank=rank(valid_res_fscore_list)
        time_mae_rank=rank(valid_time_mae_list)
        time_mse_rank=rank(valid_time_mse_list)
        # ultra_rank=act_acc_rank+act_pre_rank+act_fscore_rank+res_acc_rank+res_pre_rank+res_fscore_rank+time_mae_rank+time_mse_rank
        ultra_rank=act_acc_rank+res_acc_rank+0.4*time_mae_rank+0.4*time_mse_rank
        
        minindex=np.argmin(ultra_rank)
        tempresult=[step,test_act_acc_list[minindex],test_act_pre_list[minindex],
        test_act_fscore_list[minindex],test_res_acc_list[minindex],test_res_pre_list[minindex],test_res_fscore_list[minindex],
        test_time_mae_list[minindex],test_time_mse_list[minindex]]
        results.append(tempresult)
    train(30)




/tmp/ipykernel_17549/4047083103.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full=df_train.append(df_test).append(df_valid)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
if not os.path.exists("results/"+namedataset):
    os.mkdir("results/"+namedataset)
f=open("results/"+namedataset+"/no_task_results.csv","w")
csv_writer = csv.writer(f)
csv_writer.writerow(["fold","act_acc","act_pre","act_fscore","res_acc","res_pre","res_fscore","time_mae","time_mse"])
for i in range(5):
    csv_writer.writerow(results[i])
f.close()